In [1]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup

In [2]:
url_habr ='https://habr.com/ru/companies/'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.37",
    "Accept": "application/json, text/plain, */*"
} 

In [3]:
response = requests.get(url_habr, headers=headers)
soup = BeautifulSoup(response.text, "lxml")
pages_count = int(soup.find_all("a", class_="tm-pagination__page")[-1].text)
pages_count

17

Для проверки используется только 2стр.

In [4]:
data = pd.DataFrame(columns=['name company', 'count blogs in mart', 'badges', 'rating', 'subscribers'])
#for page in range(1, pages_count + 1):
for page in range(1, 3):
    url = f'https://habr.com/ru/companies/page{page}/'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "lxml")
    companies = soup.find_all("div", class_="tm-companies__item tm-companies__item_inlined")
    print('page:', page, '\n')
    
    for company in companies:
        # Парсинг значков
        company_badges = []
        company_link = company.find("a", class_="tm-company-snippet__title").get('href')
        response = requests.get(f'https://habr.com{company_link}', headers=headers)
        soup = BeautifulSoup(response.text, "lxml")
        badges = soup.find_all("li", class_="tm-badges__item")
        if badges == []:
                company_badges = 'Нет'
        else:
            for badge in badges:
                company_badges.append(badge.find("span").text)

        # Парсинг постов в мае  
        response = requests.get(f'https://habr.com/ru/companies/{company_link[14:-9]}/articles/', headers=headers)
        soup = BeautifulSoup(response.text, "lxml")
        company_blogs_in_mart = []
        count_blogs_in_mart = 0
        maybe_mart_blogs = True
        if soup.find_all("a", class_="tm-pagination__page") == []:
            blogs = soup.find_all("div", class_='tm-article-snippet tm-article-snippet')
            for blog in blogs:
                blog_time = blog.find("span", class_='tm-article-datetime-published').text
                if blog_time.find('мар') != -1:
                    count_blogs_in_mart += 1
                    company_blogs_in_mart.append('https://habr.com' + blog.find('a', class_='tm-title__link').get('href'))
        else:    
            pages_count = int(soup.find_all("a", class_="tm-pagination__page")[-1].text)
            for page in range(1, pages_count + 1):
                if not maybe_mart_blogs:
                    break
                response = requests.get(f'https://habr.com/kek/v2/articles/?company={company_link[14:-9]}&fl=ru&hl=ru&page={page}&perPage=20')
                JSON = response.json()
                for id_ in JSON['articleIds']:
                    date = JSON['articleRefs'][id_]['timePublished']
                    if date.find('2023-03') != -1:
                        count_blogs_in_mart += 1
                        company_blogs_in_mart.append(f'https://habr.com/ru/companies/otus/articles/{id_}/')
                    if date.find('2023-02') != -1:
                        maybe_mart_blogs = False
                        break

        # Парсинг Имени
        company_title = company.find("a", class_="tm-company-snippet__title").text

        # Парсинг кол-во рейтинга
        company_counter_rating = float(company.find("span", 
                                                    class_="tm-companies__score-counter tm-companies__score-counter_rating")
                                                    .text.strip())

        # Парсинг подписчиков
        company_subscribers = company.find("span", class_="tm-companies__score-counter tm-companies__score-counter_subscribers")\
                                                                                                                   .text.strip()
        if company_subscribers.find('K') != -1:
            company_subscribers = float(company_subscribers.replace('K', '')) * 1000
        # Вывод

        company_link = f'https://habr.com{company_link}'
        data.loc[len(data.index)] = [company_title, count_blogs_in_mart, company_badges, 
                                     company_counter_rating, company_subscribers]
        print('company:', company_title, company_link)
        print('count blogs in mart:', count_blogs_in_mart, ', links:', company_blogs_in_mart)
        print('badges:', company_badges)
        print('rating:', company_counter_rating)
        print('subscribers:', company_subscribers)
        print('\n')
        
data

page: 1 

company: RUVDS.com https://habr.com/ru/companies/ruvds/profile/
count blogs in mart: 58 , links: ['https://habr.com/ru/companies/otus/articles/725502/', 'https://habr.com/ru/companies/otus/articles/726102/', 'https://habr.com/ru/companies/otus/articles/724822/', 'https://habr.com/ru/companies/otus/articles/721686/', 'https://habr.com/ru/companies/otus/articles/723200/', 'https://habr.com/ru/companies/otus/articles/725622/', 'https://habr.com/ru/companies/otus/articles/724676/', 'https://habr.com/ru/companies/otus/articles/723538/', 'https://habr.com/ru/companies/otus/articles/725212/', 'https://habr.com/ru/companies/otus/articles/725006/', 'https://habr.com/ru/companies/otus/articles/724340/', 'https://habr.com/ru/companies/otus/articles/724868/', 'https://habr.com/ru/companies/otus/articles/724454/', 'https://habr.com/ru/companies/otus/articles/724688/', 'https://habr.com/ru/companies/otus/articles/724088/', 'https://habr.com/ru/companies/otus/articles/724304/', 'https://hab

,name company,count blogs in mart,badges,rating,subscribers
0,RUVDS.com,58,[Лучший блог компании 2020],2329.72,74000.0
1,Timeweb Cloud,49,Нет,1917.93,1200.0
2,Selectel,34,Нет,1880.78,40000.0
3,VK,12,[Лучший блог компании 2020],892.75,169000.0
4,OTUS,88,Нет,786.08,105000.0
5,Яндекс,4,[Лучший блог компании 2020],717.07,166000.0
6,ГК ЛАНИТ,4,Нет,529.47,55000.0
7,FirstVDS,23,Нет,506.69,24000.0
8,Open Data Science,2,Нет,499.40,9700.0
9,getmatch,7,Нет,466.21,990
